In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense



# Opening and Reading the csv File

In [1]:
current = r"****\OneDrive\Documents\Data Sets\Excel Files\sherlock-holm.es_stories_plain-text_advs.txt"

In [2]:
with open(current,'r',encoding='utf-8') as file:
    text = file.read()

# Tokenizing the Strings in the dataset

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1


# Creating Sequences 

In [4]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)


In [5]:
max_sequences_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen = max_sequences_len, padding = 'pre'))

In [6]:
X = input_sequences[:,:-1]
y = input_sequences[:, -1]

In [8]:
y = np.array(tf.keras.utils.to_categorical(y,num_classes = total_words))


In [9]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length= max_sequences_len -1))
model.add(LSTM(150))
model.add(Dense(total_words, activation= 'softmax'))
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2208800 (8.43 MB)
Trainable params: 2208800 (8.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [11]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X,y,epochs=100,verbose=1)

Epoch 1/100


3010/3010 [==============================] - 65s 21ms/step - loss: 6.2369 - accuracy: 0.0767
Epoch 2/100
3010/3010 [==============================] - 62s 21ms/step - loss: 5.5142 - accuracy: 0.1232
Epoch 3/100
3010/3010 [==============================] - 63s 21ms/step - loss: 5.1230 - accuracy: 0.1476
Epoch 4/100
3010/3010 [==============================] - 62s 21ms/step - loss: 4.7953 - accuracy: 0.1668
Epoch 5/100
3010/3010 [==============================] - 68s 23ms/step - loss: 4.4967 - accuracy: 0.1837
Epoch 6/100
3010/3010 [==============================] - 72s 24ms/step - loss: 4.2136 - accuracy: 0.2010
Epoch 7/100
3010/3010 [==============================] - 67s 22ms/step - loss: 3.9455 - accuracy: 0.2265
Epoch 8/100
3010/3010 [==============================] - 67s 22ms/step - loss: 3.6873 - accuracy: 0.2561
Epoch 9/100
3010/3010 [==============================] - 67s 22ms/step - loss: 3.4439 - accuracy: 0.2883
Epoch 10/100
3010/3010 [=============================

In [13]:
def Next_Word(seed_text,n):
    for _ in range(n):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list],maxlen=max_sequences_len-1,padding='pre')
        predicted = np.argmax(model.predict(token_list),axis=-1)
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word

    return seed_text

In [35]:
Next_Word("Hello", 25)

1/1 [==============================] - 0s 31ms/step


'Hello hunter had described holmes cut the cord and removed the transverse was not remembered of a broad having stood at briony lodge in the morning'